In [1]:
import yfinance as yF
import os
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import datetime
import warnings
warnings.filterwarnings("ignore")

In [2]:
#List of tickers used for the strategy.
tickers = ['XLI', 'XLB', 'XLP', 'XLK', 'XLY', 'XLV', 'XLF', 'XLE', 'XLU', 'XLC', 'XLRE', 'PDBC', 'GLD', 'TLT', 'IEF', 'TIP', 'LTPZ']

In [3]:
#Start and End date need to be at least one month greater than a year apart. 
start ="2022-01-01"
end = "2023-06-01"

In [4]:
class Data_Pull:
    def __init__(self, start, end, tickers):
        self.start_date = start
        self.end_date = end
        self.ticker_list = tickers
        
    def Data_Request(self):
        dataframe = yF.download(self.ticker_list, self.start_date, self.end_date)['Adj Close']
        return dataframe

In [5]:
class Top_Momentum:
    def __init__(self, dataframe):
        self.df = dataframe
    
    def SMA(self):
        for column in self.df:
            self.df[column+'_SMA'] = self.df[column].rolling(168).mean()
            if self.df.iloc[-1][column+'_SMA'] > self.df.iloc[-1][column]:
                self.df.drop(column, axis=1, inplace=True)
    
        regex_str = '_SMA'
        self.df = self.df.drop(self.df.columns[self.df.columns.str.contains(regex_str)], axis=1)
        global column_names
        column_names = self.df.columns.values.tolist()
    
    def Momentum_Multiple(self):
        self.df = (self.df.pct_change() +1)[1:].resample('M').prod()
        self.df = self.df - 1
        global momentum_matrix
        #Major issues with the monthly outcomes not being accurate. 
        #------
        #1 Month
        self.m1 = self.df.tail(1).copy()
        #3 Month
        self.m3 = self.df.tail(3).copy()
        #6 Month
        self.m6 = self.df.tail(6).copy()
        #9 Month
        self.m9 = self.df.tail(9).copy()
        #12 Month
        self.m12 = self.df.tail(12).copy()
        #Month 1 Perf.
        self.m1p = self.m1.copy()
        self.m1 = (self.m1p*100).round(2)
        #Month 3,6,9,12 Perf.
        df_list = self.m3, self.m6, self.m9, self.m12
        for dataframe in df_list:
            for column in dataframe:
                dataframe[column] = ((1 + dataframe[column]).cumprod()-1)
                dataframe[column] = (dataframe[column]*100).round(2)    
        #------
        #Average
        momentum_matrix = pd.concat([self.m1.tail(1), self.m3.tail(1), self.m6.tail(1), self.m9.tail(1), self.m12.tail(1)], axis=0)
        momentum_matrix = momentum_matrix.mean(axis=0).round(2)
        #Removes Assets with 0 or negative momentum
        momentum_matrix = momentum_matrix.drop(momentum_matrix[momentum_matrix <= 0].index)
        #Only selects the top 5 assets
        momentum_matrix = momentum_matrix.nlargest(n=5, keep='first')
    

In [6]:
#SMA Data
#Removes assets that are below thier 168 day or 8 month SMA. 
SMA_Data = Data_Pull(start, end, tickers)
SMA_Stock_data = SMA_Data.Data_Request()
Top_5 = Top_Momentum(SMA_Stock_data)
Top_5.SMA()

[*********************100%***********************]  17 of 17 completed


In [7]:
#Momentum
#Selects the top 5 assets based on momentum and removes assets with momentum <= 0. 
tickers = column_names
Momentum_Data = Data_Pull(start, end, tickers)
Momentum_Stock_Data = Momentum_Data.Data_Request()
Top_5 = Top_Momentum(Momentum_Stock_Data)
Top_5.Momentum_Multiple()

[*********************100%***********************]  9 of 9 completed


In [8]:
momentum_matrix

XLK    11.12
XLC     9.97
GLD     7.32
XLI     0.25
XLY     0.09
dtype: float64

In [9]:
path = ''
output_file = os.path.join(path,'USSectors_Top5.csv')
momentum_matrix.to_csv(output_file, index=True)